In [1]:
import os

In [2]:
%pwd

'e:\\ml projects\\ai_or_human_text_detection\\research'

In [3]:
os.chdir("../")

In [4]:
from pathlib import Path 
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.ai_or_human_text.utils.common import read_yaml,create_dir
from src.ai_or_human_text.constants import *

In [6]:
class ConfigManager:
    def __init__(
        self,
        config_file=CONFIG_FILE_PATH,
        schema_file=SCHEMA_FILE_PATH,
        params_file=PARAMS_FILE_PATH):

        self.config=read_yaml(config_file)
        self.schema=read_yaml(schema_file)
        self.params=read_yaml(params_file)

        create_dir([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion

        create_dir([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config. local_data_file,
            unzip_dir=config.unzip_dir)
        
        return data_ingestion_config

In [7]:
import zipfile
from urllib.request import urlretrieve
from src.ai_or_human_text.logging import logger

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config 

    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                filename,header=urlretrieve(
                    url=self.config.source_url,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded sucessfully from following header:\n {header}")

            else:
                logger.info(f"file is already exists...")
        
        except Exception as e:
            raise e
    
    def extract_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r")as zipref:
            zipref.extractall(unzip_path)

In [9]:
try:
    config=ConfigManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_file()
except Exception as e:
    raise e

[2025-08-14 12:06:29,677 : INFO : common : yaml file loaded sucessfully from config\config.yaml]
[2025-08-14 12:06:29,678 : INFO : common : yaml file loaded sucessfully from schema.yaml]
[2025-08-14 12:06:29,679 : INFO : common : yaml file loaded sucessfully from params.yaml]
[2025-08-14 12:06:29,680 : INFO : common : artifacts created sucessfully]
[2025-08-14 12:06:29,680 : INFO : common : artifacts/data_ingestion created sucessfully]
[2025-08-14 12:06:30,898 : INFO : 1988942763 : artifacts/data_ingestion/AI_generated_text_data.zip downloaded sucessfully from following header:
 Connection: close
Content-Length: 1466020
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "bf1071dab30d4437ce6312de2d64128a145e26343b234af44a8c45c4c003f948"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 13D5:34F94B